<a href="https://colab.research.google.com/github/danieljaouen/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [2]:
### YOUR CODE STARTS HERE
import pandas as pd

columns = [
  'class-name',
  'handicapped-infants',
  'water-project-cost-sharing',
  'adoption-of-the-budget-resolution',
  'physician-fee-freeze',
  'el-salvador-aid',
  'religious-groups-in-schools',
  'anti-satellite-test-ban',
  'aid-to-nicaraguan-contras',
  'mx-missile',
  'immigration',
  'synfuels-corporation-cutback',
  'education-spending',
  'superfund-right-to-sue',
  'crime',
  'duty-free-exports',
  'export-administration-act-south-africa',
]

df = pd.read_csv(
    'https://raw.githubusercontent.com/danieljaouen/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/master/module1-statistics-probability-and-inference/house-votes-84.data',
    names=columns, na_values=['?']
)

print(df.shape)
df.head()

(435, 17)


,class-name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [3]:
df = df.replace({'y': 1, 'n': 0})
df.head()

,class-name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
df.isna().sum()

class-name                                  0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [8]:
df_immigration = df.copy()
df_immigration['immigration'].dropna(inplace=True)
df_immigration['immigration'].isna().sum()

0

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [11]:
df_immigration_democrats = df_immigration[df_immigration['class-name'] == 'democrat']
df_immigration_republicans = df_immigration[df_immigration['class-name'] == 'republican']

tstat, pvalue = ttest_ind(df_immigration_democrats['immigration'].to_numpy(dtype=int), df_immigration_republicans['immigration'].to_numpy(dtype=int))
print(tstat)
print(pvalue)

0.23156844155612089
0.8169826123925337


In [12]:
df_education = df.copy()
df_education['education-spending'].dropna(inplace=True)
df_education['education-spending'].isna().sum()

0

In [14]:
df_education_democrats = df_education[df_education['class-name'] == 'democrat']
df_education_republicans = df_education[df_education['class-name'] == 'republican']

tstat, pvalue = ttest_ind(df_education_democrats['education-spending'].to_numpy(dtype=int), df_education_republicans['education-spending'].to_numpy(dtype=int))
print(tstat)
print(pvalue)

0.39250655818288804
0.6948771062928921


In [15]:
df_handicapped = df.copy()
df_handicapped['handicapped-infants'].dropna(inplace=True)
df_handicapped['handicapped-infants'].isna().sum()

0

In [16]:
df_handicapped_democrats = df_handicapped[df_handicapped['class-name'] == 'democrat']
df_handicapped_republicans = df_handicapped[df_handicapped['class-name'] == 'republican']

tstat, pvalue = ttest_ind(df_handicapped_democrats['handicapped-infants'].to_numpy(dtype=int), df_handicapped_republicans['handicapped-infants'].to_numpy(dtype=int))
print(tstat)
print(pvalue)

-0.9815786925344085
0.3268555731881082


In [17]:
df_water = df.copy()
df_water['water-project-cost-sharing'].dropna(inplace=True)
df_water['water-project-cost-sharing'].isna().sum()

0

In [18]:
df_water_democrats = df_water[df_water['class-name'] == 'democrat']
df_water_republicans = df_water[df_water['class-name'] == 'republican']

tstat, pvalue = ttest_ind(df_water_democrats['water-project-cost-sharing'].to_numpy(dtype=int), df_water_republicans['water-project-cost-sharing'].to_numpy(dtype=int))
print(tstat)
print(pvalue)

0.4585853008559582
0.646762079786876


In [19]:
df_adoption = df.copy()
df_adoption['adoption-of-the-budget-resolution'].dropna(inplace=True)
df_adoption['adoption-of-the-budget-resolution'].isna().sum()

0

In [20]:
df_adoption_democrats = df_adoption[df_adoption['class-name'] == 'democrat']
df_adoption_republicans = df_adoption[df_adoption['class-name'] == 'republican']

tstat, pvalue = ttest_ind(df_adoption_democrats['adoption-of-the-budget-resolution'].to_numpy(dtype=int), df_adoption_republicans['adoption-of-the-budget-resolution'].to_numpy(dtype=int))
print(tstat)
print(pvalue)

-0.15537826518536688
0.8765955235797003


In [0]:
# am I doing something wrong here?